# Modelling a Lens image using MCMC

In this hypothetical scenario we have an image of galaxy galaxy strong lensing and we would like to recover a model of this scene. Thus we will need to determine parameters for the background source light, the lensing galaxy light, and the lensing galaxy mass distribution. A common technique for analyzing strong lensing systems is a Markov Chain Monte-Carlo which can explore the parameter space and provide us with important metrics about the model and uncertainty on all parameters. Since caustics is differentiable we have access to especially efficient gradient based MCMC algorithms. First, we will demo a classical MCMC algorithm (using emcee) on the problem and show how for this high dimensional problem the autocorrelation length is high. Next we will show how just adding a bit of gradient information (via MALA) will significantly improve results by reducing autocorrelation length. Finally, we will demo NUTS on the problem, which is highly gradient based and is convenient in that it can be run with no tunable parameters, it's autocorrelation length is generally approximately 1. However, to achieve this, NUTS needs to run many steps internally, which ultimately means that even though it requires more tweaking to set up, MALA, is often more efficient. The best algorithm for your use case will depend on a number of factors, here you will see how caustics can play well with any sampling algorithm available, giving you lots of flexibility!

In [ ]:
import caustics
import numpy as np
import torch
import matplotlib.pyplot as plt
from matplotlib import colormaps
from matplotlib.patches import Ellipse
from scipy.stats import norm
from tqdm.notebook import tqdm

import pyro
import pyro.distributions as dist
from pyro.infer import MCMC as pyro_MCMC
from pyro.infer import NUTS as pyro_NUTS
import emcee

## Specs for the data

These are some properties of the data that aren't very interesting for the demo, it includes the size of the image, pixelscale, noise level, etc.

In [ ]:
# Data specs

background_rms = 0.005  #  background noise per pixel
exp_time = 500.0  #  exposure time (arbitrary units, flux per pixel is in units #photons/exp_time unit)
numPix = 60  #  cutout pixel size per axis
pixelscale = 0.05  #  pixel size in arcsec (area per pixel = pixel_scale**2)
fwhm = 0.05  # full width at half maximum of PSF
psf_sigma = fwhm / (2 * np.sqrt(2 * np.log(2)))
psf_type = "GAUSSIAN"  # 'GAUSSIAN', 'PIXEL', 'NONE'

cosmology = caustics.FlatLambdaCDM(name="cosmo")
cosmology.to(dtype=torch.float32)

upsample_factor = 1
quad_level = 3
thx, thy = caustics.utils.meshgrid(
    pixelscale / upsample_factor,
    upsample_factor * numPix,
    dtype=torch.float32,
)
z_l = torch.tensor(0.5, dtype=torch.float32)
z_s = torch.tensor(1.5, dtype=torch.float32)

## Build simulator forward model

Here we build the caustics simulator which will handle the lensing and generating our images for the sake of fitting. It includes a model for the lens mass distribution, lens light, and source light. We also include a simple gaussian PSF for extra realism, though for simplicity we will use the same PSF model for simulating the mock data and fitting.

In [ ]:
# Lens mass model (SIE + shear)
lens_sie = caustics.SIE(name="galaxylens", cosmology=cosmology, z_l=1.0)
lens_shear = caustics.ExternalShear(
    name="externalshear", cosmology=cosmology, x0=0.0, y0=0.0, z_l=1.0
)
lens_mass_model = caustics.SinglePlane(
    name="lensmass", cosmology=cosmology, lenses=[lens_sie, lens_shear], z_l=1.0
)

# Lens light model (sersic)
lens_light_model = caustics.Sersic(name="lenslight")

# Source light model (sersic)
source_light_model = caustics.Sersic(name="sourcelight")

# Gaussian PSF Model
psf_image = caustics.utils.gaussian(
    nx=upsample_factor * 6 + 1,
    ny=upsample_factor * 6 + 1,
    pixelscale=pixelscale / upsample_factor,
    sigma=psf_sigma,
    upsample=2,
)

# Image plane simulator
sim = caustics.LensSource(
    lens=lens_mass_model,
    lens_light=lens_light_model,
    source=source_light_model,
    psf=psf_image,
    pixels_x=numPix,
    pixelscale=pixelscale,
    upsample_factor=upsample_factor,
    z_s=2.0,
)

## Sample some mock data

Here we write out the true values for all the parameters in the model. In total there are 21 parameters, so this is quite a complex model already! We then plot the data so we can see what it is we re trying to fit.

In [ ]:
# Generate the mock data
true_params = {
    "galaxylens": {
        "x0": 0.05,
        "y0": 0.0,
        "q": 0.86,
        "phi": -0.20,
        "Rein": 0.66,
    },
    "externalshear": {"gamma_1": 0.0, "gamma_2": -0.05},
    "sourcelight": {
        "x0": 0.1,
        "y0": 0.0,
        "q": 0.75,
        "phi": 1.18,
        "n": 1.0,
        "Re": 0.1 / np.sqrt(0.75),
        "Ie": 16 * pixelscale**2,
    },
    "lenslight": {
        "x0": 0.05,
        "y0": 0.0,
        "q": 0.75,
        "phi": 1.18,
        "n": 2.0,
        "Re": 0.6 / np.sqrt(0.75),
        "Ie": 16 * pixelscale**2,
    },
}
allparams = []
for model in true_params:
    for key in true_params[model]:
        allparams.append(true_params[model][key])
allparams = torch.tensor(allparams)
print(true_params)

# simulate lens, crop extra evaluation for PSF
true_system = sim(allparams)  # simulate at high resolution

fig, axarr = plt.subplots(1, 2, figsize=(15, 8))
axarr[0].imshow(
    np.log10(true_system.detach().cpu().numpy()), cmap="inferno", origin="lower"
)
axarr[0].axis("off")
axarr[0].set_title("Mock Lens System")
torch.manual_seed(42)
shot_noise = torch.normal(
    mean=torch.zeros_like(true_system),
    std=torch.sqrt(torch.abs(true_system) / exp_time),
)
background = torch.normal(
    mean=torch.zeros_like(true_system), std=torch.tensor(background_rms)
)
variance = (torch.abs(true_system) / exp_time) + background_rms**2
obs_system = true_system + shot_noise + background
print(((obs_system - true_system) ** 2 / variance).sum().item() / 3600)
axarr[1].imshow(
    np.log10(obs_system.abs().detach().cpu().numpy()), cmap="inferno", origin="lower"
)
axarr[1].axis("off")
axarr[1].set_title("Mock Observation")
# plt.colorbar()
plt.show()

In [ ]:
def corner_plot(
    chain,
    labels=None,
    figsize=(10, 10),
    true_values=None,
    ellipse_colors="g",
):
    num_params = chain.shape[1]
    cov_matrix = np.cov(chain, rowvar=False)
    mean = np.mean(chain, axis=0)
    fig, axes = plt.subplots(num_params, num_params, figsize=figsize)
    plt.subplots_adjust(wspace=0.0, hspace=0.0)

    for i in range(num_params):
        for j in range(num_params):
            ax = axes[i, j]

            if i == j:
                x = np.linspace(
                    mean[i] - 3 * np.sqrt(cov_matrix[i, i]),
                    mean[i] + 3 * np.sqrt(cov_matrix[i, i]),
                    100,
                )
                y = norm.pdf(x, mean[i], np.sqrt(cov_matrix[i, i]))
                ax.plot(x, y, color="g")
                ax.set_xlim(
                    mean[i] - 3 * np.sqrt(cov_matrix[i, i]),
                    mean[i] + 3 * np.sqrt(cov_matrix[i, i]),
                )
                if true_values is not None:
                    ax.axvline(true_values[i], color="red", linestyle="-", lw=1)
            elif j < i:
                ax.scatter(chain[:, j], chain[:, i], color="c", s=0.1, zorder=0)
                cov = cov_matrix[np.ix_([j, i], [j, i])]
                lambda_, v = np.linalg.eig(cov)
                lambda_ = np.sqrt(lambda_)
                angle = np.rad2deg(np.arctan2(v[1, 0], v[0, 0]))
                for k in [1, 2]:
                    ellipse = Ellipse(
                        xy=(mean[j], mean[i]),
                        width=lambda_[0] * k * 2,
                        height=lambda_[1] * k * 2,
                        angle=angle,
                        edgecolor=ellipse_colors,
                        facecolor="none",
                    )
                    ax.add_artist(ellipse)

                # Set axis limits
                margin = 3
                ax.set_xlim(
                    mean[j] - margin * np.sqrt(cov_matrix[j, j]),
                    mean[j] + margin * np.sqrt(cov_matrix[j, j]),
                )
                ax.set_ylim(
                    mean[i] - margin * np.sqrt(cov_matrix[i, i]),
                    mean[i] + margin * np.sqrt(cov_matrix[i, i]),
                )

                if true_values is not None:
                    ax.axvline(true_values[j], color="red", linestyle="-", lw=1)
                    ax.axhline(true_values[i], color="red", linestyle="-", lw=1)

            if j > i:
                ax.axis("off")

            if i < num_params - 1:
                ax.set_xticklabels([])
            else:
                if labels is not None:
                    ax.set_xlabel(labels[j])
            ax.yaxis.set_major_locator(plt.NullLocator())

            if j > 0:
                ax.set_yticklabels([])
            else:
                if labels is not None:
                    ax.set_ylabel(labels[i])
            ax.xaxis.set_major_locator(plt.NullLocator())

    plt.show()

## Fit using emcee

We now model the data using emcee which handles standard Metropolis-Hastings MCMC sampling (plus a few tricks). First we need to construct a log likelihood function. In our case this is just the squared residuals, divided by the variance in each pixel. The rest is specific to the emcee implementation. Note that we must use many walkers due to the algorithm emcee uses, since this is a 21 dimensional problem we need at least 42 chains and we use 64 since that is a nice power of 2.

In [ ]:
# Make batched simulator
vsim = torch.vmap(sim)


# Log-likelihood function
def density(x):
    model = vsim(torch.as_tensor(x))
    log_likelihood_value = -0.5 * torch.sum(
        ((model - obs_system) ** 2) / variance, dim=(1, 2)
    )
    log_likelihood_value = torch.nan_to_num(log_likelihood_value, nan=-np.inf)
    return log_likelihood_value.numpy()


nwalkers = 64
ndim = len(allparams)

sampler = emcee.EnsembleSampler(nwalkers, ndim, density, vectorize=True)

x0 = allparams + 0.01 * torch.randn(nwalkers, ndim)
print("burn-in")
state = sampler.run_mcmc(x0, 100, skip_initial_state_check=True)  # burn-in
sampler.reset()
print("production")
state = sampler.run_mcmc(state, 1000)  # production

We have taken 64000 samples in this demo, in general you would want many more (each chain needs to run longer than 1000 steps in order to fully mix). Its always a good idea to plot the chains and check that they don't have any pathological features (i.e. getting frozen at one value). We subtract the mean and divide by the standard deviation of each parameter so that the chains can all be plotted together despite having very different values. Here we can see the non zero autocorrelation length for one of the chains even over 1000 steps. This indicates we should run the chains much longer, but this is just a demo.

In [ ]:
chain_mh = sampler.get_chain()
normed_chains = (chain_mh[:, 0] - np.mean(chain_mh[:, 0], axis=0)) / np.std(
    chain_mh[:, 0], axis=0
)
for i in range(chain_mh.shape[2]):
    plt.plot(normed_chains[:, i], color=colormaps["viridis"](i / chain_mh.shape[2]))
plt.title("Chain for each parameter")
plt.show()

Since the autocorrelation length is >1, we can compute an effective sample size to determine how many equivalent independent points we have drawn. As the warning suggests, in this demo we cannot compute the actual autocorrelation length, the autocorrelation length increases as we draw more samples (you can test this by changing the 1000 above to a larger number). Assuming that the autocorrelation is actually of a similar length to the chain (1000), this means we have drawn approximately 64 independent samples (one for each walker).

In [ ]:
print(
    "Autocorrelation time: ",
    np.mean(emcee.autocorr.integrated_time(chain_mh, quiet=True)),
)

We may plot the samples in a corner plot. However, we thin the samples first so that the number of points is not overwhelming to plot. As you can see in the subfigures there is still a bloby structure of the samples, suggesting that the chains were not run long enough to converge and fill the probability volume.

In this figure the green contours show the covariance matrix computed from the samples, the cyan points are the samples themselves, and the red lines are ground truth.

In [ ]:
N = chain_mh.shape[0] * chain_mh.shape[1]
fig = corner_plot(
    np.concatenate(chain_mh, axis=0)[:: int(N / 200)],
    true_values=allparams.numpy(),
)
plt.show()

# Fit with MALA sampling

Metropolis Adjusted Langevin Algorithm (MALA) sampling is the half way point between NUTS and MH, it uses gradient information to make an efficient proposal distribution for a MH step. We have written a basic implementation below for demo purposes. Essentially, one uses a random perturbation like in MH, except with a bias towards higher likelihood which comes from the gradient. Detailed balance is maintained using a MH step, so we still sample the correct distribution.

In [ ]:
def mala_sampler(
    initial_state, log_prob, log_prob_grad, num_samples, epsilon, mass_matrix
):
    """Metropolis Adjusted Langevin Algorithm (MALA) sampler with batch dimension.

    Args:
    - initial_state (numpy array): Initial states of the chains, shape (num_chains, dim).
    - log_prob (function): Function to compute the log probabilities of the current states.
    - log_prob_grad (function): Function to compute the gradients of the log probabilities.
    - num_samples (int): Number of samples to generate.
    - epsilon (float): Step size for the Langevin dynamics.
    - mass_matrix (numpy array): Mass matrix, shape (dim, dim), used to scale the dynamics.


    Returns:
    - samples (numpy array): Array of sampled values, shape (num_samples, num_chains, dim).
    """
    num_chains, dim = initial_state.shape
    samples = np.zeros((num_samples, num_chains, dim))
    x_current = np.array(initial_state)
    current_log_prob = log_prob(x_current)
    inv_mass_matrix = np.linalg.inv(mass_matrix)
    chol_inv_mass_matrix = np.linalg.cholesky(inv_mass_matrix)

    pbar = tqdm(range(num_samples))
    acceptance_rate = np.zeros([0])
    for i in pbar:
        gradients = log_prob_grad(x_current)
        noise = np.dot(np.random.randn(num_chains, dim), chol_inv_mass_matrix.T)
        proposal = (
            x_current
            + 0.5 * epsilon**2 * np.dot(gradients, inv_mass_matrix)
            + epsilon * noise
        )

        # proposal = x_current + 0.5 * epsilon**2 * gradients + epsilon * np.random.randn(num_chains, *dim)
        proposal_log_prob = log_prob(proposal)
        # Metropolis-Hastings acceptance criterion, computed for each chain
        acceptance_log_prob = proposal_log_prob - current_log_prob
        accept = np.log(np.random.rand(num_chains)) < acceptance_log_prob
        acceptance_rate = np.concatenate([acceptance_rate, accept])
        pbar.set_description(f"Acceptance rate: {acceptance_rate.mean():.2f}")

        # Update states where accepted
        x_current[accept] = proposal[accept]
        current_log_prob[accept] = proposal_log_prob[accept]

        samples[i] = x_current

    return samples

Here we run the MALA sampler after a small burn-in. We cheat a little bit and use the previous sampler to construct a mass matrix, this makes MALA more efficient but you could just as easily set the mass matrix to identity for the burn-in then use the burn-in samples to get a mass matrix, it only requires more fiddling with parameters (epsilon).

In [ ]:
def density_grad(x):
    x = torch.as_tensor(x, dtype=torch.float32)
    x.requires_grad = True
    model = vsim(x)
    log_likelihood_value = -0.5 * torch.sum(
        ((model - obs_system) ** 2) / variance, dim=(1, 2)
    )
    log_likelihood_value = torch.nan_to_num(log_likelihood_value, nan=-np.inf)
    log_likelihood_value.sum().backward()
    return x.grad.numpy()


nwalkers = 32
x0 = allparams + 0.01 * torch.randn(nwalkers, ndim)
mass_matrix = np.linalg.inv(np.cov(chain_mh.reshape(-1, ndim), rowvar=False))

chain_burnin_mala = mala_sampler(
    initial_state=x0,
    log_prob=density,
    log_prob_grad=density_grad,
    num_samples=100,
    epsilon=3e-1,
    mass_matrix=mass_matrix,
)  # burn-in

chain_mala = mala_sampler(
    initial_state=chain_burnin_mala[-1],
    log_prob=density,
    log_prob_grad=density_grad,
    num_samples=1000,
    epsilon=3e-1,
    mass_matrix=mass_matrix,
)  # production

Plotting the chains we see that they mix much better than the MH sampler, but still have some autocorrelation, as would be expected.

In [ ]:
normed_chains = (chain_mala[:, 0] - np.mean(chain_mala[:, 0], axis=0)) / np.std(
    chain_mala[:, 0], axis=0
)
for i in range(chain_mala.shape[2]):
    plt.plot(normed_chains[:, i], color=colormaps["viridis"](i / chain_mala.shape[2]))
plt.title("Chain for each parameter")
plt.show()

The autocorrelation length is better than MH as expected. Again the effective sample size can't be trusted and is probably a bit larger than the 64 value from before.

In [ ]:
print(
    "Autocorrelation time: ",
    np.mean(emcee.autocorr.integrated_time(chain_mala, quiet=True)),
)

The corner plot is much better than the MH example, we can see how the volume is filled out more with fewer gaps, suggesting we have sampled a good chunk of the space.

In [ ]:
N = chain_mala.shape[0] * chain_mala.shape[1]
fig = corner_plot(
    np.concatenate(chain_mala, axis=0)[:: int(N / 200)],
    true_values=allparams.numpy(),
)
plt.show()

## Fit using NUTS

We now model the data using NUTS. A prior is required, so we just set some extremely wide values so that we will explore just the likelihood; in general one would want to pick more informative priors. The rest is specific to the Pyro NUTS implementation!

Note, we use 25 warmup steps for Pyro, this is so it can automatically determine an appropriate "mass matrix" which helps the sampler explore much more efficiently! NUTS can also determine its own step size and build a full mass matrix. For a real analysis you would likely want to take many warm up steps so it can figure these things out accurately, afterwards the sampling will be incredibly efficient. For the sake of time in this demo, we preset a bunch of parameters and restrict what it can do, we will still get quite excellent samples.

In [ ]:
def step(model, prior):
    x = pyro.sample("x", prior)
    # Log-likelihood function
    res = model(x)
    log_likelihood_value = -0.5 * torch.sum(((res - obs_system) ** 2) / variance)
    # Observe the log-likelihood
    pyro.factor("obs", log_likelihood_value)


prior = dist.Normal(
    allparams,
    torch.ones_like(allparams) * 1e2 + torch.abs(allparams) * 1e2,
)
nuts_kwargs = {
    "jit_compile": True,
    "ignore_jit_warnings": True,
    "step_size": 2e-2,
    "full_mass": False,
    "adapt_step_size": False,
    "adapt_mass_matrix": True,
    "target_accept_prob": 0.8,
    "max_tree_depth": 8,
}

nuts_kernel = pyro_NUTS(step, **nuts_kwargs)
init_params = {"x": allparams}

# Run MCMC with the NUTS sampler and the initial guess
mcmc_kwargs = {
    "num_samples": 100,
    "warmup_steps": 25,
    "initial_params": init_params,
    "disable_progbar": False,
}

mcmc = pyro_MCMC(nuts_kernel, **mcmc_kwargs)

mcmc.run(sim, prior)

We have only taken 100 samples in this demo, in general you would want many more. Again we plot the chains and check that they look uncorrelated, everything seems fine here! There is much less structure than before, it looks like we are sampling random noise which is ideal.

In [ ]:
chain_nuts = mcmc.get_samples()["x"]
chain_nuts = chain_nuts.numpy()

normed_chains = (chain_nuts - np.mean(chain_nuts, axis=0)) / np.std(chain_nuts, axis=0)
for i in range(chain_nuts.shape[1]):
    plt.plot(normed_chains[:, i], color=colormaps["viridis"](i / chain_nuts.shape[1]))
plt.title("Chain for each parameter")
plt.show()

print(
    "Autocorrelation time: ",
    np.mean(
        emcee.autocorr.integrated_time(
            chain_nuts, has_walkers=False, tol=10, quiet=True
        )
    ),
)

As is common for NUTS sampling, the average autocorrelation time for the parameters is around 1, meaning that essentially every sample is independent. 

Finally, we show the corner plot where the samples are very well distributed as we would expect for uncorrelated samples.

In [ ]:
fig = corner_plot(chain_nuts, true_values=allparams.numpy())
plt.show()

The Pyro NUTS implementation is very convenient in that it handles the tuning of all parameters automatically in the warmup phase. In practice however, the small but efficient steps of MALA often make it very efficient, plus the rapid progress makes it easier to tweak and tune the sampler to your problem. Emcee will struggle with high dimensional problems, it was already having a hard time at 21 dimensions and gravitational lensing analysis can go much beyond 21 parameters, but it doesn't require gradient computations which may be impractical for some forward models. In the end, your specific problem will likely determine which algorithm to use.